# 0. Install required libraries, define constants, and create folders

The required libraries need only be installed once. The must be imported everytime you restart the Python kernel.

In [ ]:
pip install yt-dlp torch transformers accelerate

In [ ]:
%env HF_HOME=/cluster/user/nimeseth/.cache
%env HF_DATASETS_CACHE=/cluster/user/nimeseth/datasets

In [ ]:
VIDEO_OUTPUT_FOLDER = "tmp_videos"
AUDIO_OUTPUT_FOLDER = "tmp_audio"
TRANSCRIPTS_OUTPUT_FOLDER = "transcripts"

In [ ]:
import os

folders = [VIDEO_OUTPUT_FOLDER, AUDIO_OUTPUT_FOLDER, TRANSCRIPTS_OUTPUT_FOLDER]

for folder in folders:
    # Check if the folder exists, if not, create it
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"Folder '{folder}' created.")
    else:
        print(f"Folder '{folder}' already exists.")


# 1. Define function to download and extract YouTube videos

In [ ]:
import yt_dlp

# Takes the ID of an YouTube video and extracts the audio as a .wav file and stores it in the AUDIO_OUTPUT_FOLDER
def download_audio_from_youtube(yt_id):
    url = f"https://youtube.com/watch?v={yt_id}"
  
    # Configure yt-dlp options
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': f"{AUDIO_OUTPUT_FOLDER}/{yt_id}.%(ext)s"
    }
 
    # Download audio using yt-dlp
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    return output_file

# 2. Download and setup OpenAI Whisper v3 model for transcription

The download of the large Whisper v3 model must only be done once. This may take a few minutes.

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True)

model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device
)

# 3. Define a function to transcribe an audio file using Whisper v3

In [ ]:
def transcribe_audio_file(yt_id, language = None):

    if language is None:
        result = pipe(f"{AUDIO_OUTPUT_FOLDER}/{yt_id}.wav")
    else:
        result = pipe(f"{AUDIO_OUTPUT_FOLDER}/{yt_id}.wav", generate_kwargs={"language": language})

    with open(f"{TRANSCRIPTS_OUTPUT_FOLDER}/{yt_id}.txt", 'w', encoding='utf-8') as file:
        file.write(result["text"].strip())

# 4. Process YouTube videos from a list of IDs

In [ ]:
yt_ids = ["hntauVlkxMU", "VWGfVQaixuc"]

for yt_id in yt_ids:
    try:
        print(f"Downloading and extracting video {yt_id}")
        download_audio_from_youtube(yt_id)
    
        print(f"Transcribing video {yt_id}")
        transcribe_audio_file(yt_id, language = "german")
    
        # Delete .wav file
        os.remove(f"{AUDIO_OUTPUT_FOLDER}/{yt_id}.wav")
    
    except Exception as e:
        print(f"Error: {e}")

# 5. Process YoutTube videos from CSV file

In [ ]:
import csv

# Change the file to yours
CSV_INPUT = "csv_output/videos_from_ted_channel.csv"

with open(CSV_INPUT, mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)

    counter = 0
    max_videos = 10
    
    for row in csv_reader:

        if counter > max_videos:
            break
        
        yt_id = row[2]
        title = row[4]
        duration = float(row[6])

        if duration > 60*60:
            continue

        try:
            print(f"Downloading and extracting video {yt_id} with title '{title}'")
            download_audio_from_youtube(yt_id)
        
            print(f"Transcribing video {yt_id} with title '{title}'")
            transcribe_audio_file(yt_id, language = "english")
    
            # Delete .wav file
            os.remove(f"{AUDIO_OUTPUT_FOLDER}/{yt_id}.wav")

        except Exception as e:
            print(f"Error: {e}")

        counter = counter + 1
